# Agenda: Mask Index

- Comparisons
- Broadcasts and comparisons
- Using that to filter our series with a "boolean index" or a "mask index"
- Complex comparisons with "and" and "or"

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [7]:
s = Series([10,20,30,40,50,60,70], index=list('abcdefg'))

In [9]:
s.loc['d']

40

In [10]:
s.iloc[4]

50

In [11]:
#inside of the [], I can put a list of locations that I want to retrieve

s.loc[['a','d']]

a    10
d    40
dtype: int64

In [12]:
s.iloc[[2,5]]

c    30
f    60
dtype: int64

In [13]:
#there is anoether way that we can retrieve values, though we can pass a list of boolean values

s.loc[[True,False,False,True,True,False,True]]

a    10
d    40
e    50
g    70
dtype: int64

# Boolean/mask index

The idea here is: 
- Pass, inside of [], a list of booleans
- Whereever there is a True value, we get the value from the original se